In [ ]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import itertools



# Import Sparksession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

import warnings
warnings.filterwarnings('ignore')

spark=SparkSession.builder.appName("Data_Wrangling").getOrCreate()

In [ ]:
file_location1 = "../content/drive/MyDrive/saleforcasting/datos/2019-Dec.csv"
file_type = "csv"
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

df= spark.read.format(file_type)\
.option("inferSchema", infer_schema)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load(file_location1)

In [ ]:
df.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [ ]:
df.show()

+--------------------+----------+----------+-------------------+--------------------+-------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|  brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+-------+-------+---------+--------------------+
|2019-12-01 00:00:...|      view|   1005105|2232732093077520756|construction.tool...|  apple|1302.48|556695836|ca5eefc5-11f9-450...|
|2019-12-01 00:00:...|      view|  22700068|2232732091643068746|                null|  force| 102.96|577702456|de33debe-c7bf-44e...|
|2019-12-01 00:00:...|      view|   2402273|2232732100769874463|appliances.person...|  bosch| 313.52|539453785|5ee185a7-0689-4a3...|
|2019-12-01 00:00:...|  purchase|  26400248|2053013553056579841|computers.periphe...|   null| 132.31|535135317|61792a26-672f-4e6...|
|2019-12-01 00:00:...|      view|  20100164|2232732110089618156|    a

In [ ]:
datos_id = df.select('user_id').alias('user_id2').distinct()


In [ ]:
datos_id.to_pandas_on_spark

<bound method DataFrame.to_pandas_on_spark of DataFrame[user_id: string]>

In [ ]:
datos_id.toPandas().to_csv('/content/drive/MyDrive/saleforcasting/datos/datos_id.csv')

In [ ]:
print('El tamaño del dataset es', (df.count(), len(df.columns)))

El tamaño del dataset es (67542878, 9)


In [ ]:
spark

In [ ]:
df_e = df.dropDuplicates()

In [ ]:
df_e.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_e.columns)).show()

+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|  brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+
|         0|         0|         0|          0|      7078121|8106916|    0|      0|          21|
+----------+----------+----------+-----------+-------------+-------+-----+-------+------------+



In [ ]:
df_e = df_e.na.drop(how="any")

In [ ]:
df_e.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_e.columns)).show()

+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|         0|         0|         0|          0|            0|    0|    0|      0|           0|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+



In [ ]:
df_e.select('*').repartition(1).write.csv('/content/drive/MyDrive/saleforcasting/datos/datos_e.csv', sep=',')